In [1]:
# Obtiene la curva de distribución acumulada del modelo.

import os

import numpy as np
import pandas as pd

import xarray as xr

In [2]:
# Obtiene la curva de distribución acumulada del modelo.

import os

import numpy as np
import pandas as pd

import xarray as xr

vars = [ "Pressure", "Relative_Humidity", "Temperature",
    "Wind_Direction", "Wind_Speed", "DNI", "GHI", "UVHI",
    #"Surface_Albedo", "Precipitable_Water"
    ]
dims = ["XTIME", "south_north", "west_east"]

path_d   = "../temp/NSRDB/CDF_vars_model/"
path_m   = "../temp/TMY/CDF_vars_model/"
path_res = "../temp/TMY/CDF_vars_map/"
path_map = "../temp/TMY/qmap_vars/"
path_f   = "../temp/TMY/qmap/"

for v in vars:
    print(f"\n{v}")

    # Iteramos para todos los archivos.
    files = os.listdir( path_d + v + "/" )
    files.sort()
    if ".DS_Store" in files: files.remove(".DS_Store")
    for f in files:
        print( f"\n{f[-5:-3]}", end = "," )
        
        with xr.open_dataset(path_m + v + "/" + f) as ds_m:
            with xr.open_dataset(path_d + v + "/" + f) as ds_d:
                
                ds_m = ds_m.isel( { "south_north": slice(1, 4),
                    "west_east": slice(2, 5) } )
                ds_m["south_north"] = [0, 1, 2]
                ds_m["west_east"  ] = [0, 1, 2]

                df_d = ds_d.to_dataframe()
                df_m = ds_m.to_dataframe()

                df_m["map"] = np.interp( df_m["q_model"],
                    df_d["q_model"], df_d[v] )
                
            df_m["diff_sum"] = df_m[v] - df_m["map"]
            df_m["diff_div"] = df_m[v] / df_m["map"]

            ds_map = df_m.to_xarray()
            ds_m["map"] = ( dims, ds_map["map"].values )
            ds_m["diff_sum"] = ( dims, ds_map["diff_sum"].values )
            ds_m["diff_div"] = ( dims, ds_map["diff_div"].values )

            ds_m.to_netcdf(path_res + v + "/" + f)
            ds_m[["map"]].rename( { "map": v }
                ).to_netcdf( path_map + v + "/" + f )

            dims = ["XTIME", "south_north", "west_east"]

            ds_m["map"] = ( dims, ds_map["map"].values )
            ds_m["diff_sum"] = ( dims, ds_map["diff_sum"].values )
            ds_m["diff_div"] = ( dims, ds_map["diff_div"].values )

            ds_m.to_netcdf(f"{path_res}{v}/{f}")
            if not os.path.exists(f"{path_map}{f}/"):
                os.mkdir(f"{path_map}{f}/")
            ds_m[["map"]].rename( {"map": v} ).to_netcdf(f"{path_map}{f}/{f}")


Pressure

54,

PermissionError: [Errno 13] Permission denied: '/Users/rodrigo/Documents/Posgrado/Doctorado/Code/recurso_solar/temp/TMY/qmap_vars/Pressure/WRF_miroc_1985_2014_00054.nc'

In [ ]:
df_m["map"] = np.interp( df_m["q_model"], df_d["q_model"], df_d[v] )
df_m["diff_sum"] = df_m[v] - df_m["map"]
df_m["diff_div"] = df_m[v] / df_m["map"]

ds_map = df_m.to_xarray()
ds_m["map"] = ( dims, ds_map["map"].values )
ds_m["diff_sum"] = ( dims, ds_map["diff_sum"].values )
ds_m["diff_div"] = ( dims, ds_map["diff_div"].values )

ds_m.to_netcdf(path_res + v + "/" + files[0])
if not os.path.exists( f"{path_map}{files[0]}/" ):
    os.mkdir( f"{path_map}{files[0]}/" )
ds_m[["map"]].rename( { "map": v } ).to_netcdf(
    f"{path_map}{files[0]}/{files[0]}" )

ds_m

<xarray.Dataset> Size: 87MB
Dimensions:      (XTIME: 262958, south_north: 3, west_east: 3)
Coordinates:
  * XTIME        (XTIME) datetime64[ns] 2MB 1985-01-01 ... 2014-12-31T18:00:00
  * south_north  (south_north) int64 24B 0 1 2
  * west_east    (west_east) int64 24B 0 1 2
    XLONG        (south_north, west_east) float32 36B -99.17 -99.14 ... -99.12
    XLAT         (south_north, west_east) float32 36B 19.4 19.4 ... 19.44 19.44
Data variables:
    UVHI         (XTIME, south_north, west_east) float32 9MB 0.0 0.0 ... 32.71
    q_model      (XTIME, south_north, west_east) float64 19MB nan nan ... 1.0
    map          (XTIME, south_north, west_east) float64 19MB nan ... 0.0005987
    diff_sum     (XTIME, south_north, west_east) float64 19MB nan nan ... 32.71
    diff_div     (XTIME, south_north, west_east) float64 19MB nan ... 5.465e+04
Attributes: (12/141)
    CDI:                             Climate Data Interface version 2.1.0 (ht...
    Conventions:                     CF-1.6
    history:                         Wed Mar 20 09:49:39 2024: cdo distgrid,1...
    TITLE:                            OUTPUT FROM WRF V4.3 MODEL
    START_DATE:                      1984-09-27_00:00:00
    SIMULATION_START_DATE:           1984-01-01_00:00:00
    ...                              ...
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2
    CDO:                             Climate Data Operators version 2.1.0 (ht...